In [7]:
%load_ext autoreload
%autoreload 2
%matplotlib widget

In [8]:
import sys, os, pathlib

import numpy as np
import pandas as pd
import torch
import warp as wp
wp.config.quiet = True

sys.path.append('../..')
import project
from project.core.utils import pprint

torch.cuda.is_available()

True

In [9]:
csv_path = '2025-11-05_optimized.csv'
sel = pd.read_csv(csv_path)
sel

,dataset,subject,variant,method,u.norm,u.rms,u.rmse,u.nrmse,u.pcorr,u.scorr,...,u_isna,E_isna,corr_isna,res_isna,ucorr_isna,Ecorr_isna,res_order,u_order,E_order,failure_type
0,shapenet,wss.100f39dce7690f59efb94709f30ce0d2,2025-11-02,optimize,0.670816,1.128973,0.001666,0.001476,0.999999,0.997333,...,False,False,False,False,False,False,-0.0,-3.0,-1.0,ok
1,shapenet,wss.101354f9d8dede686f7b08d9de913afe,2025-11-02,optimize,0.622003,0.974864,0.001750,0.001795,0.999998,0.998717,...,False,False,False,False,False,False,-1.0,-3.0,-1.0,ok
2,shapenet,wss.1029fd5d71a06ea32ea98d69e91ba870,2025-11-02,optimize,0.849837,1.404901,0.002643,0.001881,0.999998,0.996461,...,False,False,False,False,False,False,0.0,-3.0,1.0,ok
3,shapenet,wss.102a6b7809f4e51813842bc8ef6fe18,2025-11-02,optimize,0.084750,0.128439,0.000404,0.003148,0.999994,0.997781,...,False,False,False,False,False,False,-1.0,-3.0,1.0,ok
4,shapenet,wss.107637b6bdf8129d4904d89e9169817b,2025-11-02,optimize,0.077659,0.257490,0.000958,0.003721,0.999993,0.973606,...,False,False,False,False,False,False,0.0,-2.0,-0.0,ok
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1110,shapenet,wss.d81f943065391492a7e760c0b399301,2025-11-02,optimize,0.054318,0.143144,0.000460,0.003214,0.999995,0.991761,...,False,False,False,False,False,False,-2.0,-2.0,-0.0,ok
1111,shapenet,wss.d8d4004791c4f61b80fa98b5eeb7036c,2025-11-02,optimize,113.014772,188.533491,0.704888,0.003740,0.999992,0.995279,...,False,False,False,False,False,False,-2.0,-2.0,-1.0,ok
1112,shapenet,wss.e8add7acb28e2505fd40810f48721e8b,2025-11-02,optimize,0.205160,0.455321,0.000704,0.001546,0.999999,0.999229,...,False,False,False,False,False,False,0.0,-3.0,-0.0,ok
1113,shapenet,wss.f6420a3a3addd027f67be63aca8832dd,2025-11-02,optimize,0.108818,0.230946,0.000418,0.001809,0.999998,0.993666,...,False,False,False,False,False,False,-0.0,-3.0,-0.0,ok


In [12]:
%autoreload
ds = project.datasets.shapenet.ShapeNetDataset(data_root='../../data/ShapeNetSem')

examples = list(ds.examples(
    subjects=list(sel.subject),
    variant='2025-11-02',
    parse_metadata=True,
    default_unit=1e-2,
    unit_policy='prefer_metadata',
    image_tag='solid_texture'
))

pprint(examples, 0)

list(len=1115)


In [14]:
import project.datasets.torch

dataset = project.datasets.torch.TorchDataset(examples)

seed = 0
all_inds = np.arange(len(dataset))
rng = np.random.default_rng(seed)
rng.shuffle(all_inds)

num_train = 1000
train_inds = all_inds[:num_train]
test_inds  = all_inds[num_train:]

train_set = torch.utils.data.Subset(dataset, train_inds)
test_set  = torch.utils.data.Subset(dataset, test_inds)

In [25]:
import project.model

model = project.model.UNet3D(
    in_channels=1,
    out_channels=1,
    n_enc_levels=2,
    n_conv_units=2,
    conv_channels=8,
    output_func='pow10'
).cuda()

n_params = project.model.count_params(model)
n_params

encoder.level0.conv_block.unit0.conv.weight        216	(8, 1, 3, 3, 3)
encoder.level0.conv_block.unit0.norm.weight        8	(8,)
encoder.level0.conv_block.unit0.norm.bias          8	(8,)
encoder.level0.conv_block.unit1.conv.weight        1728	(8, 8, 3, 3, 3)
encoder.level0.conv_block.unit1.norm.weight        8	(8,)
encoder.level0.conv_block.unit1.norm.bias          8	(8,)
encoder.level1.conv_block.unit0.conv.weight        3456	(16, 8, 3, 3, 3)
encoder.level1.conv_block.unit0.norm.weight        16	(16,)
encoder.level1.conv_block.unit0.norm.bias          16	(16,)
encoder.level1.conv_block.unit1.conv.weight        6912	(16, 16, 3, 3, 3)
encoder.level1.conv_block.unit1.norm.weight        16	(16,)
encoder.level1.conv_block.unit1.norm.bias          16	(16,)
decoder.level0.conv_block.unit0.conv.weight        5184	(8, 24, 3, 3, 3)
decoder.level0.conv_block.unit0.norm.weight        8	(8,)
decoder.level0.conv_block.unit0.norm.bias          8	(8,)
decoder.level0.conv_block.unit1.conv.weight      

19361

In [26]:
batch_size = 1
x = torch.zeros((batch_size, 1, 256, 256, 256), dtype=torch.float32, device='cuda')
n_activs = project.model.count_activations(model, x)
n_activs

input                                    16777216	(1, 1, 256, 256, 256)	torch.float32
encoder.level0.conv_block.unit0.conv     134217728	(1, 8, 256, 256, 256)	torch.float32
encoder.level0.conv_block.unit0.norm     134217728	(1, 8, 256, 256, 256)	torch.float32
encoder.level0.conv_block.unit1.conv     134217728	(1, 8, 256, 256, 256)	torch.float32
encoder.level1.conv_block.unit0.conv     33554432	(1, 16, 128, 128, 128)	torch.float32
encoder.level1.conv_block.unit1.conv     33554432	(1, 16, 128, 128, 128)	torch.float32
decoder.level0.conv_block.unit1.conv     134217728	(1, 8, 256, 256, 256)	torch.float32


620756992

In [27]:
model_mib = (n_activs + n_params) * 2 * 4 / 2**20 # MiB
model_mib

4736.1477127075195

In [28]:
import project.datasets.torch

loader_kws = dict(
    batch_size=1,
    shuffle=True,
    num_workers=4,
    pin_memory=True,
    collate_fn=project.datasets.torch.collate_fn
)

train_loader = torch.utils.data.DataLoader(train_set, **loader_kws)
test_loader = torch.utils.data.DataLoader(train_set, **loader_kws)

In [29]:
import torch.optim
optimizer = torch.optim.AdamW(model.parameters(), lr=5e-3)
optimizer

AdamW (
Parameter Group 0
    amsgrad: False
    betas: (0.9, 0.999)
    capturable: False
    differentiable: False
    eps: 1e-08
    foreach: None
    fused: None
    lr: 0.005
    maximize: False
    weight_decay: 0.01
)

In [30]:
import project.evaluation
evaluator = project.evaluation.Evaluator()

In [33]:
import project.training
import warp as wp
wp.config.quiet = True
solver_kws = dict(reg_weight=1e-4)
trainer = project.training.Trainer(model, optimizer, train_loader, test_loader, evaluator, solver_kws=solver_kws)

In [34]:
trainer.train(1)

Start training loop
Epoch 0/1
Start test phase
Test batch 1/1000
Start forward
Model forward
PDE forward


Warp CUDA error 101: invalid device ordinal (in function radix_sort_reserve_internal, /builds/omniverse/warp/warp/native/sort.cu:51)
Warp CUDA error 2: out of memory (in function wp_alloc_device_async, /builds/omniverse/warp/warp/native/warp.cu:657)
Warp CUDA error 101: invalid device ordinal (in function radix_sort_pairs_device, /builds/omniverse/warp/warp/native/sort.cu:102)


TypeError: _validate_scalar_field() got an unexpected keyword argument 'check_positive'